In [1]:
import random
from util_class.ImageStruct import ImageStruct
import heapq
import torch

In [2]:
annotation_file, tensor_file = \
      './data/anotation_new.csv', './image_tensor/model_image_2023-07-08/resnet50_tensor.pt'
images = ImageStruct(annotation_file, tensor_file)

In [3]:
src_index = random.randint(0, len(images))
s_img_tensor, s_img_path, s_category, s_caption = images.get(src_index)
s_category, s_caption

('ネックレス × ベージュ',
 ' アー・ヴェ・ヴェ（a.v.v）のメタルパーツワントップネックレス。◆人気のワントップネックレス◆きれいめにもカジュアルにも合わせやすいメタルパーツのネックレスです')

In [4]:
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得
class CaptionEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")
  def forward(self, x):
    x = self.bert(x)
    x = x.last_hidden_state
    # print(x.shape)
    x = x[:,0,:] 
    # print(x.shape)
    return x

model_name =  './model/model_caption_2023-07-08.pth'
caption_model = CaptionEncoder().to(device)
caption_model.load_state_dict(torch.load(model_name))
caption_model.eval()

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
input_text = ["デニム ",
               "シャツ",
               "レザー",
               "スカート",]
ids = tokenizer.batch_encode_plus(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=256, add_special_tokens=True).input_ids
ids = ids.to(device)
s_caption_tensor = caption_model(ids).to('cpu')

In [14]:
from util.category import get_item_id

In [28]:

heaps = [[] for i in range(len(input_text))]
MAX = 300000
for i in range(MAX):
    if (i % 1000 == 0):
        print(f"{i * 100 / MAX} %")
    t_img_tensor, t_img_path, t_category, _ = images.get(i)
    t_id = get_item_id(t_img_path)
    for j in range(len(input_text)):
        heap = heaps[j]
        heap_ids = [get_item_id(i[1]) for i in heap]
        if (t_id in heap_ids):
            continue
        # print(s_category, t_category)
        # if (s_category != t_category):
        #     continue
        dist = torch.dist(s_caption_tensor[j], t_img_tensor, p=2)
        heapq.heappush(heap, (-dist.item(), t_img_path))
        if (len(heap) > 10):
            heapq.heappop(heap)


0.0 %
0.3333333333333333 %
0.6666666666666666 %
1.0 %
1.3333333333333333 %
1.6666666666666667 %
2.0 %
2.3333333333333335 %
2.6666666666666665 %
3.0 %
3.3333333333333335 %
3.6666666666666665 %
4.0 %
4.333333333333333 %
4.666666666666667 %
5.0 %
5.333333333333333 %
5.666666666666667 %
6.0 %
6.333333333333333 %
6.666666666666667 %
7.0 %
7.333333333333333 %
7.666666666666667 %
8.0 %
8.333333333333334 %
8.666666666666666 %
9.0 %
9.333333333333334 %
9.666666666666666 %
10.0 %
10.333333333333334 %
10.666666666666666 %
11.0 %
11.333333333333334 %
11.666666666666666 %
12.0 %
12.333333333333334 %
12.666666666666666 %
13.0 %
13.333333333333334 %
13.666666666666666 %
14.0 %
14.333333333333334 %
14.666666666666666 %
15.0 %
15.333333333333334 %
15.666666666666666 %
16.0 %
16.333333333333332 %
16.666666666666668 %
17.0 %
17.333333333333332 %
17.666666666666668 %
18.0 %
18.333333333333332 %
18.666666666666668 %
19.0 %
19.333333333333332 %
19.666666666666668 %
20.0 %
20.333333333333332 %
20.66666666666

In [29]:
from util.save_combined_image import save_combined_image

from datetime import datetime
import os

# 現在の日付を取得します
now = datetime.now()

# YYYY-MM-DD形式で日付を出力します
formatted_date = now.strftime("%m-%d")
for i in range(len(input_text)):
    try:
        os.mkdir(f'./result/{formatted_date}/{input_text[i]}')
    except FileExistsError:
        pass

    save_combined_image(s_img_path, [j[1] for j in heaps[i]], f'./result/{formatted_date}/{input_text[i].strip()}/normal.png')

FileNotFoundError: [Errno 2] No such file or directory: './result/07-09/デニム /normal.png'